In [ ]:
import pandas as pd

In [ ]:
ruta = "/content/solveTFM.txt"

with open(ruta, "r") as archivo:
  contenido = archivo.read()

texto = contenido.split("VAR X",1)[1]
texto = "1" + texto.split("1",1)[1]
texto = texto.rsplit("EPS",1)[0]

lineas = texto.split("\n")

nummed = int(len(lineas)/(3*28))

matrix = [[[0 for _ in range(28)] for _ in range(3)] for _ in range(nummed)]

for linea in lineas:
  linea = linea[0:8] + linea[22]
  if linea[8] != "1":
    linea = linea[0:8] + "0"
  pos1 = int(linea[0:2])
  pos2 = int(linea[3])
  pos3 = int(linea[5:7])
  matrix[pos3-1][pos2-1][pos1-1] = int(linea[8])

df1 = pd.DataFrame(index=range(3), columns=range(28))

for i in range(3):
  for j in range(28):
    escribe = " "
    for k in range(nummed):
      if matrix[k][i][j] == 1:
        escribe = escribe + " " + str(k+1) + "; "
    df1.at[i,j] = escribe

df1.index = df1.index + 1
df1.columns = df1.columns + 1


#df1.to_excel("/content/solveTFM.xlsx")

In [ ]:

df2 = pd.DataFrame(index=range(nummed), columns=range(11))
df2.index = df2.index + 1
df2.columns = ['Horas mensuales', 'Horas semana 1', 'Horas semana 2',
               'Horas semana 3', 'Horas semana 4', 'Mañanas L-V', 'Tardes L-V',
               'Noches L-J', 'Sabados-domingos 16h', 'Noches V-D', 'Dias libres']


In [ ]:
for i in range(nummed):
  hs = [0, 0, 0, 0]; sd = 0; n = 0; nf = 0; dl = 0
  m = 0; t = 0
  for j in range(28):
    aux_dl = 1
    str_to_search = " " + str(i+1) + "; "
    if j%7 < 4:
      if str_to_search in df1.at[1, j+1]:
        m = m+1; aux_dl = 0; hs[j//7] = hs[j//7] + 7
      if str_to_search in df1.at[2, j+1]:
        t = t+1; aux_dl = 0; hs[j//7] = hs[j//7] + 7
      if str_to_search in df1.at[3, j+1]:
        n = n+1; aux_dl = 0; hs[j//7] = hs[j//7] + 10
      dl = dl + aux_dl
    elif j%7 == 4:
      if str_to_search in df1.at[1, j+1]:
        m = m+1; aux_dl = 0; hs[j//7] = hs[j//7] + 7
      if str_to_search in df1.at[2, j+1]:
        t = t+1; aux_dl = 0; hs[j//7] = hs[j//7] + 7
      if str_to_search in df1.at[3, j+1]:
        nf = nf+1; aux_dl = 0; hs[j//7] = hs[j//7] + 10
      dl = dl + aux_dl
    else:
      if str_to_search in df1.at[1, j+1]:
        sd = sd+1; aux_dl = 0; hs[j//7] = hs[j//7] + 14
      if str_to_search in df1.at[3, j+1]:
        nf = nf+1; aux_dl = 0; hs[j//7] = hs[j//7] + 12
      dl = dl + aux_dl
  df2.loc[i+1] = [sum(hs)] + hs + [m, t, n, sd, nf, dl]

In [ ]:
rutaexcel = "/content/solveTFM.xlsx"

with pd.ExcelWriter(rutaexcel) as writer:
    # Guardar el DataFrame df1 en la hoja 'Hoja1'
    df1.to_excel(writer, sheet_name='Hoja1')

    # Guardar el DataFrame df2 en la hoja 'Hoja2'
    df2.to_excel(writer, sheet_name='Hoja2')